# Run Daily Space-Time LISA

* Software dependencies

In [1]:
import tools
import pandas as pd
import numpy as np
import pysal as ps
import multiprocessing as mp
from sqlalchemy import create_engine
import geopandas as gpd

/Users/dani/anaconda/envs/pydata/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


* Data dependencies

In [2]:
db_link ='/Users/dani/AAA/LargeData/adam_cell_phone/a10.db'
shp_link = '../data/a10/a10.shp'
# To be created in the process:
ashp_link = '/Users/dani/Desktop/a10_agd_maxp.shp'

engine = create_engine('sqlite:///'+db_link)

* Read data in

In [3]:
%%time
a10 = pd.read_sql_query('SELECT gridcode, date_time, trafficerlang '
                         'FROM data ',
                         engine, parse_dates=['date_time'])

months = a10['date_time'].apply(lambda x: str(x.year) + '-' + str(x.month))
hours = a10['date_time'].apply(lambda x: str(x.hour))

order = ps.open(shp_link.replace('.shp', '.dbf')).by_col('GRIDCODE')
areas = pd.Series([poly.area for poly in ps.open(shp_link)], \
                  index=order)
areas = areas * 1e-6 # Sq. Km

CPU times: user 41.8 s, sys: 3.44 s, total: 45.2 s
Wall time: 52.7 s


* MaxP

This step removes an area with no data and joins very small polygons to adjacent ones with density as similar as possible. This is performed through an aggregation using the Max-P algorithm

In [4]:
shp = gpd.read_file(shp_link).set_index('GRIDCODE')

overall = a10.groupby('gridcode').mean()
overall['area (Km2)'] = areas
overall['erldens'] = overall['trafficerlang'] / overall['area (Km2)']
overall = gpd.GeoDataFrame(overall, geometry=shp['geometry'], crs=shp.crs)\
             .dropna()
    
        # W
wmxp = ps.queen_from_shapefile(shp_link, idVariable='GRIDCODE')
wmxp.transform = 'R'
wmxp.transform = 'O'
# Polygon `49116` does not have data. Remove.
wmxp = ps.w_subset(wmxp, [i for i in wmxp.id_order if i!=49116])

# Information matrix with hourly average day
x = a10.assign(hour=hours).groupby(['gridcode', 'hour'])\
       .mean()['trafficerlang']\
       .unstack()\
       .reindex(wmxp.id_order)
# Areas for the MaxP
mxp_a = overall.loc[wmxp.id_order, 'area (Km2)'].values

In [5]:
%%time
np.random.seed(1234)
mxp = ps.Maxp(wmxp, x.values, 0.05, mxp_a, initial=1000)
labels = pd.Series(mxp.area2region).apply(lambda x: 'a'+str(x))

CPU times: user 8.31 s, sys: 20.7 ms, total: 8.33 s
Wall time: 8.38 s


* Aggregate polygons

In [6]:
aggd = overall.groupby(labels).sum()
aggd['erldens'] = aggd['trafficerlang'] / aggd['area (Km2)']

ag_geo = overall.groupby(labels)['geometry'].apply(lambda x: x.unary_union)
aggd_shp = gpd.GeoDataFrame(aggd, geometry=ag_geo, crs=overall.crs)
aggd_shp.reset_index().to_file(ashp_link)

ag_a10 = a10.assign(hour=hours, month=months)\
    .set_index('gridcode')\
    .assign(labels=labels)\
    .groupby(['month', 'hour', 'labels', 'date_time'])[['trafficerlang']].sum()\
    .reset_index()

* $ST-W$

In [7]:
    # W
aw = ps.queen_from_shapefile(ashp_link, idVariable='index')

aw.transform = 'R'
aw.transform = 'O'
    
    # Space-Time W
ats = ag_a10['hour'].unique().shape[0]
%time astw = tools.w_stitch_single(aw, ats)
astw.transform = 'R'

CPU times: user 45 ms, sys: 3.51 ms, total: 48.5 ms
Wall time: 48.5 ms


* Expand areas

In [8]:
aareas = aggd_shp.reset_index().set_index('index')
astw_index = pd.Series(astw.id_order, \
                         index=[i.split('-')[1] for i in astw.id_order], \
                        name='astw_index')
astareas = aareas.reset_index()\
                .join(astw_index, on='index')\
                .drop('index', axis=1)\
                .set_index('astw_index')\
                [['area (Km2)']]

* Reshape for daily runs

In [45]:
daily = ag_a10.drop('month', axis=1)\
              .assign(h_gc=ag_a10['hour']+'-'+ag_a10['labels'])\
              .join(astareas, on='h_gc')\
              .assign(date=ag_a10['date_time'].apply(lambda x: str(x.date())))\
              .set_index(['date', 'hour', 'labels'])
daily['erldens'] = daily['trafficerlang'] / daily['area (Km2)']

* Run in parallel

In [145]:
permutations = 1

g = daily.groupby(level='date')
tasks = [(i, astw, astareas, permutations, id) for id, i in g]
#pool = mp.Pool(mp.cpu_count())
%time tasks = map(tools.child_lisa, tasks)
lisa_clusters = pd.concat(tasks, axis=1)

#lisa_clusters.to_csv('../data/lisa_clusters_%ip.csv'%permutations)ss

CPU times: user 1min 16s, sys: 227 ms, total: 1min 16s
Wall time: 1min 16s
